In [ ]:
!pip install langchain
!pip install pinecone-client
!pip install pypdf
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import pinecone
import os
import sys

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 3.4 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
loader = PyPDFDirectoryLoader("/content/area")
data = loader.load()

In [ ]:
data[0]

Document(page_content=" \n \nMUHAMMAD  ISHAQ  \n+92-331-8201165  • ishaqjan619@gmail.com  \n• https://www.linkedin.com/in/ishaq -jan-165a37a0/  • Age:  24 \n \n Education   \nThe National  University  of Computer  and Emerging  Sciences  FAST  Islamabad  \nMaster  of Computer  Science  2023  \nHITEC  UNIVERSITY,  TAXILA  Taxila,  Pakistan  \nBachelor  of Computer  Science  (CGPA  3.13/4.00)  2017 -2021  \nPUNJAB  COLLEGE  ISLAMABAD  Islamabad,  Pakistan  \nIntermediate  2015 -2017  \n Professional  Experience   \n \nAMAL  ACADEMY  Lahore,  Pakistan  \nEducation  startup  funded  by Stanford  University  that teaches  professional  skills  to students and  corporations.  \n– Communication : Practiced the concept of active listening and developed various effective  \ncommunication  skills  like maintaining  calm  under  pressure,  politeness,  friendliness,  and knowing  \nwhen  to ask questions.  \n– Skills  development : Improved  writing  and reading  skill.  Strengthened  speaking  s

In [ ]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)

In [ ]:
len(text_chunks)

17

In [ ]:
!pip install --upgrade google-api-python-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 79.5 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.84.0
    Uninstalling google-api-python-client-2.84.0:
      Successfully uninstalled google-api-python-client-2.84.0


In [ ]:
os.environ['GOOGLE_API_KEY'] = ''

In [ ]:
!pip install google.generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.7/267.7 kB 13.4 MB/s eta 0:00:00


In [ ]:
embeddings=GooglePalmEmbeddings()
query_result = embeddings.embed_query("Hello World")

In [ ]:
embeddings

GooglePalmEmbeddings(client=<module 'google.generativeai' from '/usr/local/lib/python3.10/dist-packages/google/generativeai/__init__.py'>, google_api_key=None, model_name='models/embedding-gecko-001')

In [ ]:
print("Length",len(query_result))

Length 768


In [ ]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', '')

In [ ]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "lama2" # put in the name of your pinecone index here

In [ ]:
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)


In [ ]:
docsearch=Pinecone.from_existing_index(index_name,embeddings)

In [ ]:
query="Hi"

In [ ]:
docs = docsearch.similarity_search(query, k=3)


In [ ]:
llm = GooglePalm(temperature=0.1)


In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())


In [ ]:
prompt_template="""Act as AI resume shortlisting agent.\
You are provided a pdf resume\
Scan the resume and extract the main keywords from it.\
Give summary of the resume whether the person is talented or not.

{context}

Question: {question}
Helpful answer in markdown:"""

In [ ]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [ ]:
query = "What is work experience"


In [ ]:
qa.run(query)

'The candidate has 3 months of work experience as an apprentice at Interloop Limited.'

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 7.9 MB/s eta 0:

In [ ]:
import sys
import gradio as gr
conversation_history = []

custom_css = """
    .gr-textbox input {
        height: 200px;
    }
"""
def answer_question(user_input):
    if user_input == 'exit':
        print('Exiting')
        sys.exit()
    if user_input == '':
        return 'Please enter a question.'

    result = qa.run(user_input)
    answer = result  # Since result is assumed to be a string in this code
    return answer


iface = gr.Interface(
    fn=answer_question,
    inputs=gr.inputs.Textbox(type="text", label="Input Question"),
    outputs=gr.outputs.Textbox(type="text", label="Answer"),
    title="ResuMate: Smart Resume Analysis and Candidate Evaluation",
    description="Ask a question and get an answer. Type 'exit' to quit.",
    theme="huggingface",
    layout="vertical",
    css=custom_css,  # Apply the custom CSS styles
)

iface.launch(share=True)

<ipython-input-27-a198260618d3>:25: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Textbox(type="text", label="Input Question"),
<ipython-input-27-a198260618d3>:25: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Textbox(type="text", label="Input Question"),
<ipython-input-27-a198260618d3>:25: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Textbox(type="text", label="Input Question"),
<ipython-input-27-a198260618d3>:26: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs=gr.outputs.Textbox(type="text", label="Answer"),
/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:695: UserWarning: Cannot load huggingface. Caught Exception: The

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://00af102769e92ce1f6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
